### Some things about bitwise operations.

x & y

Does a "bitwise and". Each bit of the output is 1 if the corresponding bit of x AND of y is 1, otherwise it's 0. 

example:

2&2 = 2
       
        2 ---> 0010
          and
        2 ---> 0010
              ------ 
               0010 = 2      
2&1 = 0
        
        2 ---> 0010
          and
        1 ---> 0001
              ------ 
               0000 = 1


x | y
    
Does a "bitwise or". Each bit of the output is 0 if the corresponding bit of x AND of y is 0, otherwise it's 1. 
    
2&1 = 3
        
        2 ---> 0010
           or
        1 ---> 0001
              ------ 
               0011 = 3


To have the octants (0-7) in binary numbers in mind: 

    0000 = 0        0001 = 1        0010 = 2         0011 = 3

    0100 = 4        0101 = 5        0110 = 6         0111 = 7


## Constructing the tree

Let's start to define the problem and our rules to build the tree. Consider there are $n=100$ particles randomly scatterd in the domain $x$, $y$, $z$ $\in$ $\left[ 0, 1 \right]$, each of them is a source and target. To contain all the particles, we define a **root** cubic cell centered at $(0.5,0.5, 0.5)$ with a side length of $1$. For a cubic cell, the radius is the half of side length, thus the root cell's radius $r_r$ is $0.5$. Then we choose $n_{crit}=10$ as the threshold to split a cell.

In [1]:
import numpy
from treecode_helper import Point, Particle

In [2]:
n = 100          # number of particles
particles = [ Particle(m=1.0/n) for i in range(n) ]

n_crit = 10      # max number of particles in a single cell

### Defining the class : Cell

Since each non-empty cell is an instance which has some same properties (eg. cell's center coordinates, its parent, its children, its multipole, particles inside if it's a leaf), we first need to define a class for a cell, and we call this class "Cell". Every cell is an instance of **Class** "**Cell**", and all the instances are stored in a **list** called "**cells**". Therefore, the root cell is "cells[0]". For those who have not been exposed to object-oriented programming in python, check [this](http://www.tutorialspoint.com/python/python_classes_objects.htm) out as a quick guide.

<img src="image/cell_class.png">

The figure above shows the "content" of a cell element:
* $x_c$, $y_c$, $z_c$, $r_c$: center coordinates and radius give the geometry of the cell
* a leaf of a cell is a particle stored in the cell, each leaf corresponds to a particle index from $0$ to $n-1$, and $n_{leaf}$ is number of leaves in the cell
* **parent** is the index (in the cells list) of the corresponding parent. 
* **child** is the array that contains the indices (in the cells list) of the corresponding children.
* **nchild** is an 8-bit binary number, each digit represents one of the eight child octants. $1$ denotes non-empty child cell, and $0$ denotes empty child cell in that octant. For example: nchild=00010100 means the current cell only has non-empty child in the fourth and sixth octant.
* **multipole**: array of $10$ multipole terms of the cell

In [3]:
class Cell():
    '''
    The class for a cell
    
    Arguments
    ----------
        n_crit: maximum number of particles in a leaf cell.
        
    Attributes
    ----------
        nleaf (int): number of leaves in the cell.
        leaf (array of int): array of leaves indices.
        nchild (int): 8-bit binary number, used to keep
                      track of the empty child cells
        child (array of int): array of children indices.
        parent (int): index of parent cell. 
        x, y, z (float): coordinates of the cell's center.
        r (float): radius of the cell (half of the length for cubic cell)
        multipole (array of float): array of multipoles' cell.
    '''
    
    def __init__(self, n_crit):
        
        self.nleaf = 0
        self.leaf = numpy.zeros(n_crit, dtype=numpy.int)
        self.nchild = 0
        self.child = numpy.zeros(8, dtype=numpy.int)
        self.parent = 0
        self.x = self.y = self.z = 0.
        self.r = 0. 
        self.multipole= numpy.zeros(10, dtype=numpy.float)

In [4]:
#Let's generate the root cell

root = Cell(n_crit)
root.x, root.y, root.z = 0.5, 0.5, 0.5
root.r = 0.5

If we want to add more than 10 particles then we need to create a child by splitting the root cell. "Adding a cell" is going to be a dependency of "splitting a cell".

The new child will be append at the end of the cells list.

The relationships between parent and children are:
    
* $r_{child}$ = $\frac{1}{2}r_{parent}$ for a cubic cell.
* $x_{c_{c}}$, $y_{c_{c}}$, $z_{c_{c}}$ can be determined by its $octant$ and its parent's coordinates $x_{c_{p}}$, $y_{c_{p}}$, $z_{c_{p}}$.

We need to establish a mutual reference between the parent and the child in the cells list. Consider a new child is created in the parent's $5th$ octant. We assign the new child's index to the parent by "parent.child[$4$]=index_child", and assign the parent's index to the new child by "child.parent=index_parent". Don't forget the 8-bit binary marker "nchild". Since the new child is at $5th$ octant, the fifth digit from the right should be changed from "0" to "1". Recall that we always manipulate the binary number with bit shift.


In [5]:
def add_child(octant, p, cells, n_crit):
    '''
    Appends a cell at the end of cells list as a child of p,
    initializes the center and radius of the child cell c, 
    and establishes mutual reference between the child and parent p.
    
    Arguments:
        octant: reference to the corresponding octant in 3D structure.
        p: is the index (in the cells list) of the corresponding parent.
        cells: the list of cells.
        n_crit: maximum number of particles in a leaf cell.
    '''
    
    #create a new cell instance and append it to cells list
    cells.append(Cell(n_crit))

    #the last element of the cells list is the new child
    c = len(cells) - 1
    
    #geometric relationship between parent and child
    cells[c].r = cells[p].r / 2.
    cells[c].x = cells[p].x + cells[p].r * ((octant & 1) * 2 -1)
    cells[c].y = cells[p].y + cells[p].r * ((octant & 2) - 1)
    cells[c].z = cells[p].z + cells[p].r * ((octant & 4) / 2 -1)
    
    #establish mutual reference in the cells list
    
    cells[c].parent = p
    cells[p].child[octant] = c
    
    # in this line we update the nchild the | operation results    
    # in a adding 1 in the place of the bit where we just add a child.
    cells[p].nchild = (cells[p].nchild | (1<<octant))     
    print('+++ cell {} is created as a child of cell {}'.format(c,p)) 

### Splitting a cell

The "split" method will be recursive because there is a probability that  after splitting a cell, all the particles are reallocated to the same child. In this scenario, we have to recursively split the child cell again until all the cells satisfy the rule $n_{crit}=10$.

Consider now we put $>$10 particles in the root cell. So it is time to split the root. In addition to create new child cells, we have also to "settle" down these particles. We loop over the them, and each particle is located in a certain octant from 0 to 7. If there is not a child cell in that octant, then we create one. If there is a child cell already created, we just put this particle in that child. Finally, we check if the number of particles in that child reaches $n_{crit}$, if yes, we split it recursively. 

<img src="image/split_cell.png">

In [ ]:
def split_cell(particles, p, cells, n_crit):
    '''
    Loop in parent p's leaves and reallocare the particles to subcells. 
    If a subcell has not been created in that octant, it creates one 
    using add_child. If the subcell nleaf exceeds n_crit, split the
    subcell "c" recursively.
    
    Arguments:
    ----------
        particles: the list of particles.
        p: is the index (in the cells list) of the corresponding parent.
        cells: the list of cells.
        n_crit: maximum number of particles in a leaf cell.
    '''
    
    print('=========start the splitting of cell {}========='.format(p))
    
    # loop over the particles in the parent cell that you want to split
    
    for l in cells[p].leaf:
        # finding the particle's octant 
        octant = (particles[l].x > cells[p].x) + \
                 ((particles[l].y > cells[p].y) << 1) + \
                 ((particles[l].z > cells[p].z) << 2)   
    
        # if there is not a child cell in the particle's octant, then create one
        if not cells[p].nchild & (1 << octant):
            add_child(octant, p, cells, n_crit)
    
        # reallocate the particle in the child cell
        c = cells[p].child[octant]
        cells[c].leaf[cells[c].nleaf] = l
        cells[c].nleaf += 1
        
        print('>>>particle {} is reallocated in cell {}'.format(l, c))
        
        # check if the child reach n_crit, if yes ---> split cell
        if cell[c].nleaf >= n_crit: #nleaf starts on 0 that's why the =
            split_cell(particles, c, cells, n_crit)
    print('=========end the splitting of cell {}========='.format(p))